In [3]:
import pandas as pd
import numpy as np
import plotly.express as px

df_analise = pd.read_csv('dolar0902.csv')

In [10]:
df = df_analise[:-100000]#.loc[ : '2022-02-09 10']
df

,time,bid,ask,last,volume,IFR5,IFR9,IFR14
0,2022-02-09 09:30:00,5276.5,5277.0,5276.5,1,40.072808,44.576042,46.563722
1,2022-02-09 09:30:00,5276.5,5277.0,5277.0,1,56.828692,54.099051,52.725012
2,2022-02-09 09:30:00,5276.5,5277.0,5277.0,89,56.828692,54.099051,52.725012
3,2022-02-09 09:30:00,5276.5,5277.0,5276.5,6,39.550026,44.435956,46.506096
4,2022-02-09 09:30:00,5276.5,5277.0,5277.0,3,56.197587,53.733099,52.535219
...,...,...,...,...,...,...,...,...
109278,2022-02-09 12:14:48,5297.5,5298.0,5297.5,1,42.811845,44.062904,44.565971
109279,2022-02-09 12:14:48,5297.5,5298.0,5297.5,18,42.811845,44.062904,44.565971
109280,2022-02-09 12:14:48,5297.5,5298.0,5298.0,2,59.596742,53.420122,50.690070
109281,2022-02-09 12:14:48,5297.5,5298.0,5297.5,1,43.600605,44.959196,45.300508


In [60]:
def percentil(data, up=90, down=10):
    x = data
    array_x = np.array(x)
    percentil_up = np.percentile(array_x, up)
    percentil_down = np.percentile(array_x, down)
    
    return percentil_up, percentil_down

In [61]:
n1,n2 =percentil(df['IFR5'], 98.5, 1.5)
#line_color='rgb(231,107,243)
n1, n2

(73.21458889907109, 28.30642024819653)

In [63]:
fig = px.line(df, y=df['IFR5'])
fig.add_hline(y=n1)
fig.add_hline(y=n2)
fig.show()

In [47]:
# for aninhado com listas
profit = []
flag = 0
s = []
dif = []
listsoma = []
resultado = 0
df['diferenca'] = df['last'].diff()
soma = 0
valor_diferencial_atual = 0

for i in range(len(df)):
    rv = df.IFR14[i]
    profit.append(np.nan)
    dif.append(df.diferenca[i])
    s.append(soma)
    valor_diferencial_atual = df.diferenca[i]

    if df['IFR14'][i] <= 40 and flag != 1: # condição de entrada

        flag = 1
        profit.pop()
        profit.append('start')
        soma = 0
        listsoma.clear()

    soma += df.diferenca[i]
    listsoma.append(valor_diferencial_atual)
    acumuladolista = np.cumsum(listsoma)
    resultado = acumuladolista[-1]



    if resultado >= 10.0 and flag == 1: #condição de saida
        profit.pop()
        profit.append('gain')
        listsoma.clear()
        resultado = 0
        acumuladolista = 0
        flag = 0
        soma = 0
    
    if resultado <= -10.0 and flag == 1: #condição de saida
        profit.pop()
        profit.append('loss')
        listsoma.clear()
        resultado = 0
        acumuladolista = 0
        flag = 0
        soma = 0

df['profit'] = profit

<ipython-input-47-b8fdb4ee8719>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-47-b8fdb4ee8719>:52: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [48]:
profit_df = pd.DataFrame(profit)
profit_df = profit_df.dropna()
profit_df.columns = ['profit']
profit_df = profit_df[profit_df['profit'] != 'start']

In [49]:
profit_df.value_counts()

profit
gain      3
loss      2
dtype: int64

In [50]:
fig = px.pie(profit_df, names='profit') 
fig.show()

In [23]:
import pandas as pd
import numpy as np
import plotly.express as px

df = pd.read_csv('D:\Estudos gerais\dolinha.csv')

# for aninhado com listas
profit = []
flag = 0
s = []
dif = []
listsoma = []
resultado = 0
df['diferenca'] = df.close.diff()
soma = 0
valor_diferencial_atual = 0
men = []
mep = []


for i in range(len(df)):
    rv = df.real_volume[i]
    profit.append(np.nan)
    dif.append(df.diferenca[i])
    s.append(soma)
    valor_diferencial_atual = df.diferenca[i]
    men.append(np.nan)
    mep.append(np.nan)

    if df['real_volume'][i] <= 5000.0 and flag != 1:  # condição de entrada

        flag = 1
        profit.pop()
        profit.append('start')
        soma = 0
        listsoma.clear()
        mep.pop()
        men.pop()
        mep.append(0)
        men.append(0)

    if flag == 1:
        soma += df.diferenca[i]
        listsoma.append(valor_diferencial_atual)
        acumuladolista = np.cumsum(listsoma)
        resultado = acumuladolista[-1]

    #colocar condição de comprado e vendido
    if resultado > 0 and flag == 1 and profit[-1] != 'start':
        mep.pop()
        mep.append(resultado) # exposição positiva
        men.pop()
        men.append(0)

    if resultado < 0 and flag == 1 and profit[-1] != 'start':
        men.pop()
        men.append(resultado) # exposição negativa
        mep.pop()
        mep.append(0)

    if resultado >= 5.0 and flag == 1:  # condição de saida
        profit.pop()
        profit.append('gain')
        listsoma.clear()
        resultado = 0
        acumuladolista = 0
        flag = 0
        soma = 0

    if resultado <= -5.0 and flag == 1:  # condição de saida
        profit.pop()
        profit.append('loss')
        listsoma.clear()
        resultado = 0
        acumuladolista = 0
        flag = 0
        soma = 0

df['profit'] = profit
df['MEP'] = mep
df['MEN'] = men

df = df.fillna(0)
df['EXP'] = df['MEP'] - ( - df['MEN'])

df

,time,open,high,low,close,tick_volume,spread,real_volume,diferenca,profit,MEP,MEN,EXP
0,2021-12-07 09:04:00,5697.0,5702.0,5697.0,5700.5,543,0,13624,0.0,0,0.0,0.0,0.0
1,2021-12-07 09:05:00,5701.0,5702.5,5698.5,5701.0,1516,500,14821,0.5,0,0.0,0.0,0.0
2,2021-12-07 09:06:00,5700.5,5703.5,5700.0,5702.0,795,500,6977,1.0,0,0.0,0.0,0.0
3,2021-12-07 09:07:00,5702.5,5702.5,5700.0,5701.5,579,500,5538,-0.5,0,0.0,0.0,0.0
4,2021-12-07 09:08:00,5701.5,5702.0,5700.0,5700.0,422,500,4953,-1.5,start,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,2021-12-07 18:25:00,5649.0,5652.0,5646.0,5647.0,280,500,2523,-2.0,0,2.5,0.0,2.5
562,2021-12-07 18:26:00,5647.0,5647.5,5645.5,5646.0,92,500,594,-1.0,0,1.5,0.0,1.5
563,2021-12-07 18:27:00,5647.0,5647.5,5645.5,5645.5,49,500,541,-0.5,0,1.0,0.0,1.0
564,2021-12-07 18:28:00,5646.0,5649.0,5646.0,5648.5,103,500,1138,3.0,0,4.0,0.0,4.0


In [37]:
gain = 5
loss = -5
fig = px.line(df, y=df['EXP'])
fig.add_hline(y=gain, line_color='green')
fig.add_hline(y=0, line_color='black')
fig.add_hline(y=loss, line_color='red')
fig.add_hrect(y0=0, y1=gain, line_width=0, fillcolor="green", opacity=0.2)
fig.add_hrect(y0=0, y1=loss, line_width=0, fillcolor="red", opacity=0.2)
fig.show()


'fig.add_vline(x=2.5, line_width=3, line_dash="dash", line_color="green")\nfig.add_hrect(y0=0.9, y1=2.6, line_width=0, fillcolor="red", opacity=0.2)\nfig.show()'

In [ ]:
# testar area
fig = px.area(df, x="year", y="pop", color="continent", line_group="country")
fig.show()

In [38]:
#df = pd.read_csv('dolar0902.csv')

In [41]:
#df = df[:-100000]#.loc[ : '2022-02-09 10']

In [74]:
df = pd.read_csv('D:\Estudos gerais\dolinha.csv')

In [121]:
import pandas as pd
import numpy as np
import plotly.express as px

df = pd.read_csv('D:\Estudos gerais\dolinha.csv')

gain = 10.0
loss = -10.0

# for aninhado com listas
profit = []
flag = 0
s = []
dif = []
listsoma = []
resultado = 0
df['diferenca'] = df.close.diff()
soma = 0
valor_diferencial_atual = 0
men = []
mep = []
final = []
exp = []
vfinal = 0

for i in range(len(df)):
    rv = df.real_volume[i]
    profit.append(np.nan)
    dif.append(df.diferenca[i])
    s.append(soma)
    valor_diferencial_atual = df.diferenca[i]
    men.append(0)
    mep.append(0)
    final.append(0)
    exp.append(0)

    if df['real_volume'][i] <= 5000.0 and flag != 1:  # condição de entrada

        flag = 1
        profit.pop()
        profit.append('start')
        soma = 0
        listsoma.clear()
        mep.pop()
        men.pop()
        mep.append(0)
        men.append(0)

    if flag == 1:
        soma += df.diferenca[i]
        listsoma.append(valor_diferencial_atual)
        acumuladolista = np.cumsum(listsoma)
        resultado = acumuladolista[-1]

    #colocar condição de comprado e vendido
    if resultado > 0 and flag == 1 and profit[-1] != 'start':
        mep.pop()
        mep.append(resultado) # exposição positiva
        men.pop()
        men.append(0)

    if resultado < 0 and flag == 1 and profit[-1] != 'start':
        men.pop()
        men.append(resultado) # exposição negativa
        mep.pop()
        mep.append(0)

    if resultado >= gain and flag == 1:  # condição de saida
        profit.pop()
        profit.append('gain')
        listsoma.clear()
        resultado = 0
        acumuladolista = 0
        flag = 0
        soma = 0

    if resultado <= loss and flag == 1:  # condição de saida
        profit.pop()
        profit.append('loss')
        listsoma.clear()
        resultado = 0
        acumuladolista = 0
        flag = 0
        soma = 0

    if flag == 1:
        exp.pop()
        exp.append(men[-1] - (- mep[-1]))

    if profit[-1] == 'gain':
        #final.pop()
        vfinal += mep[-1]


    if profit[-1] == 'loss':
        #final.pop()
        vfinal += men[-1]

    final.pop()
    final.append(vfinal)

df['profit'] = profit
df['MEP'] = mep
df['MEN'] = men
df['Final'] = final

df

,time,open,high,low,close,tick_volume,spread,real_volume,diferenca,profit,MEP,MEN,Final
0,2021-12-07 09:04:00,5697.0,5702.0,5697.0,5700.5,543,0,13624,NaN,NaN,0.0,0.0,0.0
1,2021-12-07 09:05:00,5701.0,5702.5,5698.5,5701.0,1516,500,14821,0.5,NaN,0.0,0.0,0.0
2,2021-12-07 09:06:00,5700.5,5703.5,5700.0,5702.0,795,500,6977,1.0,NaN,0.0,0.0,0.0
3,2021-12-07 09:07:00,5702.5,5702.5,5700.0,5701.5,579,500,5538,-0.5,NaN,0.0,0.0,0.0
4,2021-12-07 09:08:00,5701.5,5702.0,5700.0,5700.0,422,500,4953,-1.5,start,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
561,2021-12-07 18:25:00,5649.0,5652.0,5646.0,5647.0,280,500,2523,-2.0,NaN,2.5,0.0,-19.5
562,2021-12-07 18:26:00,5647.0,5647.5,5645.5,5646.0,92,500,594,-1.0,NaN,1.5,0.0,-19.5
563,2021-12-07 18:27:00,5647.0,5647.5,5645.5,5645.5,49,500,541,-0.5,NaN,1.0,0.0,-19.5
564,2021-12-07 18:28:00,5646.0,5649.0,5646.0,5648.5,103,500,1138,3.0,NaN,4.0,0.0,-19.5


In [125]:
df.head(10)

,time,open,high,low,close,tick_volume,spread,real_volume,diferenca,profit,MEP,MEN,Final
0,2021-12-07 09:04:00,5697.0,5702.0,5697.0,5700.5,543,0,13624,NaN,NaN,0.0,0.0,0.0
1,2021-12-07 09:05:00,5701.0,5702.5,5698.5,5701.0,1516,500,14821,0.5,NaN,0.0,0.0,0.0
2,2021-12-07 09:06:00,5700.5,5703.5,5700.0,5702.0,795,500,6977,1.0,NaN,0.0,0.0,0.0
3,2021-12-07 09:07:00,5702.5,5702.5,5700.0,5701.5,579,500,5538,-0.5,NaN,0.0,0.0,0.0
4,2021-12-07 09:08:00,5701.5,5702.0,5700.0,5700.0,422,500,4953,-1.5,start,0.0,0.0,0.0
5,2021-12-07 09:09:00,5700.5,5700.5,5696.0,5698.5,1249,500,16722,-1.5,NaN,0.0,-3.0,0.0
6,2021-12-07 09:10:00,5699.0,5700.0,5696.0,5699.5,796,500,10180,1.0,NaN,0.0,-2.0,0.0
7,2021-12-07 09:11:00,5699.5,5700.0,5696.5,5699.0,466,500,6114,-0.5,NaN,0.0,-2.5,0.0
8,2021-12-07 09:12:00,5699.0,5699.0,5691.0,5691.5,1392,500,17132,-7.5,loss,0.0,-10.0,-10.0
9,2021-12-07 09:13:00,5691.5,5693.0,5688.5,5689.5,1623,500,16111,-2.0,NaN,0.0,0.0,-10.0


In [123]:
df['profit'].value_counts()

start    13
loss      7
gain      5
Name: profit, dtype: int64

In [124]:
gain = max(df['Final']) * 1.10
loss = min(df['Final']) * 1.10
fig = px.line(df, y=df['Final'])
fig.add_hline(y=gain, line_color='green')
fig.add_hline(y=0, line_color='black')
fig.add_hline(y=loss, line_color='red')
fig.add_hrect(y0=0, y1=gain, line_width=0, fillcolor="green", opacity=0.2)
fig.add_hrect(y0=0, y1=loss, line_width=0, fillcolor="red", opacity=0.2)
fig.show()